In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import scipy 
from neurosim.models.ssr import StateSpaceRealization as SSR, gen_random_model
from dca_research.kca import calc_mmse_from_cross_cov_mats
import matplotlib.pyplot as plt

/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
from tqdm import tqdm
import torch

In [4]:
sys.path.append('../..')
from utils import calc_loadings

### Testing LQG invariant-related identities

In [39]:
# Solve using the matrix sign function. The advantage of this is that one gets all 4 matrices of interest in one go

# See: Positive and negative solutions of dual Riccati equations by matrix sign function iteration
def sgn(H):
    Z = H
    for i in range(500):
        Z = 1/2 * (Z + np.linalg.inv(Z))
    return Z

def solve_are_sgn(A, B, C):

    # Hamiltonian matrix associated with the LQR problem. The transpose is
    # associated with the filtering problem
    H = np.block([[A, -B @ B.T], [-C.T @ C, -A.T]])
    Z = sgn(H)
    Z11 = Z[0:A.shape[0], 0:A.shape[0]]
    Z12 = Z[0:A.shape[0], A.shape[0]:]
    Z21 = Z[A.shape[0]:, 0:A.shape[0]]
    Z22 = Z[A.shape[0]:, A.shape[0]:]

    Pp = -1 * scipy.linalg.pinv(np.block([[Z12], [Z22 + np.eye(A.shape[0])]])) @ np.block([[Z11 + np.eye(A.shape[0])], [Z21]])
    Pm = -1 * scipy.linalg.pinv(np.block([[Z12], [Z22 - np.eye(A.shape[0])]])) @ np.block([[Z11 - np.eye(A.shape[0])], [Z21]])

    # Take the transpose of the Hamiltonian matrix and proceed as before
    Z = sgn(H).T
    Z11 = Z[0:A.shape[0], 0:A.shape[0]]
    Z12 = Z[0:A.shape[0], A.shape[0]:]
    Z21 = Z[A.shape[0]:, 0:A.shape[0]]
    Z22 = Z[A.shape[0]:, A.shape[0]:]

    Qp = -1 * scipy.linalg.pinv(np.block([[Z12], [Z22 + np.eye(A.shape[0])]])) @ np.block([[Z11 + np.eye(A.shape[0])], [Z21]])
    Qm = -1 * scipy.linalg.pinv(np.block([[Z12], [Z22 - np.eye(A.shape[0])]])) @ np.block([[Z11 - np.eye(A.shape[0])], [Z21]])

    return Pp, Pm, Qp, Qm


In [105]:
Pp, Pm, Qp, Qm = solve_are_sgn(A, B, C)

In [108]:
np.linalg.cond(Pp)

1.5789733726932968e+16

In [109]:
np.linalg.cond(Pm)

2.4137913499767225

In [113]:
np.allclose(np.linalg.inv(Pm), -1 * Qp)

True

In [114]:
np.allclose(np.linalg.inv(Qm), -1 * Pp)

False

In [115]:
# One gets the right answer when the condition numbers are reasonable...

In [6]:
# Discrete time testing
A, B, C = gen_random_model(20)
A = 1/2 * (A + A.T)
#A = np.diag(np.random.uniform(0, 1, size=(20,)))

In [7]:
np.linalg.eigvals(A)

array([-0.96019059, -0.6924917 , -0.6074098 ,  0.77639008,  0.7313606 ,
        0.64905692,  0.57270791,  0.4932683 , -0.44861674, -0.35492999,
       -0.32065521, -0.33719855,  0.3667119 ,  0.27059249,  0.18798358,
        0.10707903, -0.12402855, -0.07472079,  0.0079528 , -0.02754021])

In [8]:
P1 = scipy.linalg.solve_discrete_are(A, B, C.T @ C, np.eye(B.shape[1]))
# Dual solution
P2 = scipy.linalg.solve_discrete_are(A.T, C.T, B @ B.T, np.eye(C.shape[0]))

In [13]:
Q1 = solve_dare(A, B, C.T @ C, np.eye(B.shape[1]))
Q1m = solve_dare(A, B, C.T @ C, np.eye(B.shape[1]), False)

Q2 = solve_dare(A.T, C.T, B @ B.T, np.eye(C.shape[0]))
Q2m = solve_dare(A.T, C.T, B @ B.T, np.eye(C.shape[0]), False)

In [244]:
np.trace(Q1)

23.96079954242218

In [16]:
np.allclose(np.linalg.inv(Q1m), -1*P2)

True

In [319]:
A, B, C = gen_random_model(20)

In [320]:
ssr = SSR(A, B, C)

In [330]:
ccm = ssr.autocorrelation(5)

In [322]:
from dca.cov_util import calc_cov_from_cross_cov_mats

In [335]:
calc_mmse_from_cross_cov_mats(torch.tensor(ccm).float(), proj=torch.eye(ccm.shape[1]).float())

tensor(20.)

In [336]:
ssr.solve_min_phase()

In [290]:
X = np.block([[covf.numpy(), covpf.numpy().T], [covpf.numpy(), covp.numpy()]])

In [276]:
np.linalg.eigvals(covf.numpy())

array([2.32110202, 1.58520366])

In [277]:
np.linalg.eigvals(covp.numpy())

array([4.7620994 , 4.73554397, 3.95362946, 3.90252817, 3.21494589,
       3.16970859, 2.69407875, 2.64382498, 1.85947067, 2.39583849,
       2.34771529, 2.32758591, 2.32135343, 2.06495108, 2.2485509 ,
       2.20993624, 2.14318295, 2.1417027 , 1.72135523, 1.50840663,
       1.3169601 , 1.16886422, 1.14774686, 1.14580719, 1.12512732,
       1.13924207, 1.13645881, 1.13444905, 1.09242932, 1.08197266,
       1.05989906, 1.0527829 , 1.03637703, 1.03723791, 1.04083439,
       1.04137276, 1.04781372, 1.04802378])

In [279]:
torch.trace(covf - torch.chain_matmul(covpf.t(), torch.inverse(covp), covpf))

tensor(-1.8538, dtype=torch.float64)

In [287]:
np.linalg.cond(covp.numpy())

4.5949488271275865

In [286]:
covpf.t() @ torch.inverse(covp) @ covpf

tensor([[ 3.0336, -0.5099],
        [-0.5099,  2.7265]], dtype=torch.float64)

In [281]:
torch.trace(covf - covpf.t() @ torch.inverse(covp) @ covpf)

tensor(-1.8538, dtype=torch.float64)

In [284]:
np.trace(covf.numpy() - covpf.numpy().T @ np.linalg.inv(covp.numpy()) @ covpf.numpy())

-1.8537728993631153

In [299]:
# Does forward Riccati equation converge to MMSE
ccm = ssr.autocorrelation(20)
cov = calc_cov_from_cross_cov_mats(ccm)

In [311]:
np.concatenate([c[np.newaxis, :] for c in ccm]).shape

(20, 2, 2)

In [304]:
# MMSE forward
def mmse_forward(ccm, proj=None):

    if proj is not None:
        ccm_proj

    T = ccm.shape[0] - 1
    N = ccm.shape[-1]
    cov = calc_cov_from_cross_cov_mats(ccm)
    cov_proj = calc_cov_from_cross_cov_mats(np.concatenate([(proj.T @ c @ proj)[np.newaxis, :] for c in ccm])

    covf = cov[-N:, -N:]
    covp = cov[:T*N, :T*N]
    covpf = cov[:T*N, -N:]
    covfp = cov[-N:, :T*N]

    return covf - covfp @ np.linalg.inv(covp) @ covpf


def mmse_reverse(ccm):
    pass    

In [305]:
mmse_forward(ccm)

array([[ 1.88057021, -0.38751228],
       [-0.38751228,  1.63696645]])

In [307]:
ssr.solve_min_phase()

In [17]:
# Does the antistabilizing solution to the Riccati equation coincide with the solution of the Riccati equation obtained from a backwards
# Markovian realization of the process?

# One immediate consequence of the Kailath formula is that the forward and reverse time Kalman filter parameters should coincide.

In [157]:
A, B, C = gen_random_model(20, 10, cont=True)
# B = C.T

In [158]:
Pi = scipy.linalg.solve_continuous_lyapunov(A, -B @ B.T)

In [159]:
Pp, Pm, Qp, Qm = solve_are_sgn(A, B, C)

In [160]:
np.allclose(Qm, -1*np.linalg.inv(Pp))

True

In [161]:
np.linalg.cond(Pm)

2.213196988879976

In [162]:
np.allclose(-A, -A - B @ B.T @ np.linalg.inv(Pi))

False

In [171]:
Q1 = solve_are(A.T, C.T, B @ B.T, np.eye(C.shape[0]), stable=True)

In [176]:
Q2 = solve_are((-A - B @ B.T @ np.linalg.inv(Pi)).T, C.T, B @ B.T, np.eye(C.shape[0]), stable=True)

In [172]:
np.linalg.eigvals(Q1)

array([0.37810058, 0.36822253, 0.33790802, 0.32270506, 0.300653  ,
       0.29239621, 0.1708391 , 0.28015369, 0.27312249, 0.17790005,
       0.18521241, 0.19358801, 0.20305031, 0.25809415, 0.21077485,
       0.2451703 , 0.23787772, 0.23383626, 0.22500763, 0.22561057])

In [177]:
np.linalg.eigvals(Q2)

array([0.37678043, 0.36530915, 0.33857327, 0.32329659, 0.17082345,
       0.30140423, 0.17730941, 0.29075257, 0.28078575, 0.27382403,
       0.18554753, 0.1930167 , 0.20329504, 0.25765317, 0.21120329,
       0.24535055, 0.23822446, 0.23321985, 0.22534598, 0.22686252])

In [180]:
np.linalg.eigvals(Pp)b

array([3.07868345e-01, 2.99673619e-01, 2.62684288e-01, 2.59082437e-01,
       2.30970964e-01, 2.26881340e-01, 2.17022759e-01, 1.84176358e-01,
       2.00077744e-01, 1.96047785e-01, 9.06777542e-03, 6.29964175e-03,
       5.00721777e-03, 4.29488689e-03, 2.75048337e-03, 1.66799693e-03,
       8.47813473e-04, 4.25302854e-05, 1.04944233e-04, 2.07173131e-04])

In [181]:
# Conclusion is that we need to normalize the reverse time parameterization to obtain the adjoint state system
# Numerically verify 2 things:
# (1) The acausal Kalman filter Riccati solution coincides with the empirical MMSE
# (2) The acausal filtering problem for the adjoint state coincides with the solution of the forward time regulator riccati equation

In [182]:
# First task: Does discrete time MMSE converge to continuous time riccati solution as we make the timestep increasingly smaller?

In [183]:
# Back up: Does our implementation of mmse_from_cross_cov_mats work in the discrete time case?

In [215]:
from dca_research.cov_util import calc_mmse_from_cross_cov_mats

In [192]:
A, B, C = gen_random_model(20)

In [198]:
ssr = SSR(A, B, C)
ssr.solve_min_phase()
ccm = ssr.autocorrelation(10)

In [194]:
calc_mmse_from_cross_cov_mats(torch.tensor(ccm))

tensor(20.0000, dtype=torch.float64)

In [199]:
np.trace(ssr.P - ssr.Pmin)

20.000000000000007

In [200]:
# Now do projected version
A, B, C = gen_random_model(20, 2)

In [210]:
ssr = SSR(A, B, C)
ssr_ambient = SSR(A, B, C=np.eye(A.shape[0]))
ssr.solve_min_phase()
ccm = ssr_ambient.autocorrelation(10)

In [211]:
calc_mmse_from_cross_cov_mats(torch.tensor(ccm), proj=torch.tensor(C.T))

> /home/akumar/nse/DCA_research/dca_research/kca.py(46)calc_mmse_from_cross_cov_mats()
     44         covf = cross_cov_mats[0]
     45         pdb.set_trace()
---> 46         covpf = torch.cat(ccm_proj2)
     47 
     48     else:



tensor(34.3742, dtype=torch.float64)

In [212]:
np.trace((ssr.P - ssr.Pmin))

34.373397726939075

In [216]:
# Next task: Does the (discrete time) acausal Riccati equation induced by the backwards model coincide with the empirical mmse?
A, B, C = gen_random_model(20)
# Construct acausal parameters:
ssr_fwd = SSR(A, B, C)
Ar = ssr_fwd.P @ A @ np.linalg.inv(ssr_fwd.P)
Br = ssr_fwd.P - Ar @ ssr_fwd.P @ Ar.T

# Projection
V = scipy.stats.ortho_group.rvs(20)[:, 0:2].T

ssr_bkwd_ambient = SSR(Ar, Br, C)
ccm = ssr_bkwd_ambient.autocorrelation(10)
ssr_bkwd = SSR(Ar, Br, V)
ssr_bkwd.solve_min_phase()

In [217]:
calc_mmse_from_cross_cov_mats(torch.tensor(ccm), proj=torch.tensor(V.T))

tensor(133.7017, dtype=torch.float64)

In [218]:
np.trace(ssr_bkwd.P - ssr_bkwd.Pmin)

133.70133716047124

In [6]:
# Next: Does discrete time MMSE converge to the solution of the continuous time Riccati equation as we let delta t -> 0?
deltat = np.logspace(-3, 0, 10)[::-1]
nt = np.array([10, 25, 50, 100])
diff = np.zeros((deltat.size, nt.size))

A, B, C = gen_random_model(20, cont=True)
Pcont = scipy.linalg.solve_continuous_lyapunov(A, -B @ B.T)
V = scipy.stats.ortho_group.rvs(20)[:, 0:2].T

Q = scipy.linalg.solve_continuous_are(A.T, V.T, B @ B.T, np.eye(V.shape[0]))

for i, dt in enumerate(deltat):
    for j, n in enumerate(nt):        
        ccm = np.array([scipy.linalg.expm(A * j * dt) @ Pcont for j in range(n)]) 
        m1 = calc_mmse_from_cross_cov_mats(torch.tensor(ccm), proj=torch.tensor(V.T)).detach().numpy()
        diff[i, j] = np.trace(Q - m1)

In [ ]:
# Then, code up the modification to LQGCA in which we normalize by the state variance

### Testing whether acausal filtering covariance is the inverse of the LQR grammian

In [ ]:
# To test this, one calculates the solution of the Riccati equation associated with filtering of the co-state, and then scales by Pi^{-1} Q Pi^{-1}^T

In [1]:
sys.path.append('/home/akumar/nse/network_control')

In [21]:
from state_space import gen_random_model
from state_space import ContinuousSSR as CSSR

In [30]:
A, B, C = gen_random_model(10, 5, 5, continuous=True)

In [31]:
Pi = scipy.linalg.solve_continuous_lyapunov(A, -B @ B.T)

In [32]:
Q = scipy.linalg.solve_continuous_are(A.T, (C @ Pi).T, np.linalg.inv(Pi) @ B @ B.T @ np.linalg.inv(Pi), np.eye(C.shape[0]))

In [41]:
Pbp, Pbm, Qbp, Qbm = solve_are_sgn(A.T, (C @ Pi).T, (np.linalg.inv(Pi) @ B).T)

In [42]:
P = scipy.linalg.solve_continuous_are(A, B, C.T @ C, np.eye(B.shape[1]))

In [46]:
np.linalg.norm(np.linalg.inv(P) - Pi @ Pbp @ Pi.T)

398.5252188860716

In [47]:
# Simulate a discrete time system, flip the direction of time, and compare to Riccati solutions

In [58]:
from dca_research.lqg import calc_mmse_from_cross_cov_mats
import torch

In [149]:
A, B, C = gen_random_model(20, 18, 5)

In [135]:
C.shape

(5, 20)

In [169]:
ssr = SSR(A, B=B, C=C)
ssr2 = SSR(A=A, B=B, C=np.eye(20))

In [151]:
ccm = ssr2.autocorrelation(20)

In [170]:
ssr.solve_min_phase()

In [171]:
np.trace(ssr.P - ssr.Pmin)

29.040384172401914

In [174]:
np.trace(Q)

29.040385037250346

In [173]:
Q = scipy.linalg.solve_discrete_are(A.T, C.T, B @ B.T, np.zeros((C.shape[0], C.shape[0])))

In [188]:
P = scipy.linalg.solve_discrete_are(A, B, C.T @ C, np.eye(B.shape[1]))

In [183]:
ccm_rev = np.array([c.T for c in ccm])

In [184]:
np.trace(calc_mmse_from_cross_cov_mats(torch.tensor(ccm_rev), torch.tensor(C.T)))

28.67491046048867

6715.517264960441